# What makes Happiness?

For a long time, people have been interested in what makes us happy and how we can improve well-being in society. One big question is whether having more money or wealth actually makes people happier, or if there are other things that matter more. We often hear the saying “money can’t buy happiness,” but the real answer is a bit more complicated than that.

In this data story, we will explore how happiness relates to different economic and social factors using information from the World Happiness Report 2019 and World Development Indicators. This report looks at how happy people are in different countries and compares that with demographic variables like GDP per capita, Gross National Income (GNI), the gini index, unemployment rates, education levels, and life expectancy.

We want to find out if richer countries really have happier people, and if so, how strong this connection is. But we will also look beyond money to see how things like having a job, going to school, and living a long, healthy life affect happiness. For example, being unemployed might make people less happy even if their country is wealthy, and having a good education could improve well-being in ways that money alone can’t.

First, we will compare happiness scores with income indicators like GDP per capita and GNI. Then, we will analyze how unemployment rates relate to happiness. After that, we will look at the gini index, education and life expectancy to see how these factors could affect happiness scores across countries.

By comparing these different aspects, we hope to better understand what really contributes to happiness around the world. This will help us see whether the saying “money buys happiness” really holds. And if it doesn’t hold, we could find out what does make happiness.


**Money does buy happiness**

It is often proposed that greater national wealth leads to higher levels of happiness among citizens. To see wether they are correlated, we analysed how average happiness scores relate to three key economic indicators: Gross Domestic Product (GDP), Gross National Income (GNI), and the Gini Index (Income inequality).



The trendlines suggest that countries with higher GDP and GNI often report higher happiness levels. This supports the general belief that wealth leads to happiness due to the fact that wealth increases access to basic needs, and social stability, factors that are positively associated with well-being and happiness. However, the relationship is not linear, and several wealthy countries show only moderate happiness scores, this suggests other factors are still extremely relevant and happiness cannot be predicted by looking at wealth alone. Correlation does not imply causation, and the observed pattern may be influenced by unmeasured factors.

The third plot, Happiness vs Gini Index, presents a different insight. Within this dataset, a negative correlation is visible, with happiness tending to decline as income inequality (measured by the Gini Index) increases. . In other words, even among wealthier nations, greater inequality is associated with lower average happiness. This aligns with findings by researchers such as Oishi et al., (2011), who in their research found that americans reported being happier on average in years where the national income inequality was less. This suggests income inequality might be a factor which has impact on the happiness . However income inequality may be acting as a substitute for how people perceive the fairness in society and the level of social mobility people have. Happiness is also a subjective measurement, and cultural differences in how inequality is perceived can vary widely, potentially affecting how people respond in surveys.

The plots suggest that while wealth does matter, its distribution may be even more of a factor. Factors such as income equality are likely important to how people will respond to questions about their happiness. 

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

df = pd.read_csv('final.csv')

df['Happiness_rank'] = df['Happiness'].rank(method='min', ascending=False).astype(int)
total_countries = df['Country'].nunique()

df_gini = df.dropna(subset=['Gini_index', 'Happiness', 'Country', 'Happiness_rank'])
df_gdp  = df.dropna(subset=['GDP_current_usd', 'Happiness', 'Country', 'Happiness_rank'])
df_gni  = df.dropna(subset=['GNI_current_usd', 'Happiness', 'Country', 'Happiness_rank'])

x_range_gini = [df_gini['Gini_index'].min(), df_gini['Gini_index'].max()]
x_range_gdp  = [df_gdp['GDP_current_usd'].min(), 5e11]
x_range_gni  = [df_gni['GNI_current_usd'].min(), 5e11]
y_range      = [df['Happiness'].min(), df['Happiness'].max()]

def add_trend_trace(x, y, color, visible=True):
    x_vals = x.values
    y_vals = y.values
    slope, intercept = np.polyfit(x_vals, y_vals, 1)
    x_trend = np.linspace(min(x_vals), max(x_vals), 100)
    y_trend = slope * x_trend + intercept
    
    return go.Scatter(
        x=x_trend,
        y=y_trend,
        mode='lines',
        line=dict(color=color, dash='dash'),
        name='Trend Line',
        visible=visible,
        hoverinfo='skip'
    )

trace_gini = go.Scatter(
    x=df_gini['Gini_index'],
    y=df_gini['Happiness'],
    mode='markers',
    name='Gini Index',
    visible=True,
    text=df_gini['Country'],
    customdata=df_gini['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Gini Index: %{x:.1f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_gdp = go.Scatter(
    x=df_gdp['GDP_current_usd'],
    y=df_gdp['Happiness'],
    mode='markers',
    name='GDP per Capita',
    visible=False,
    text=df_gdp['Country'],
    customdata=df_gdp['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "GDP per Capita: $%{x:,.0f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_gni = go.Scatter(
    x=df_gni['GNI_current_usd'],
    y=df_gni['Happiness'],
    mode='markers',
    name='GNI per Capita',
    visible=False,
    text=df_gni['Country'],
    customdata=df_gni['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "GNI per Capita: $%{x:,.0f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trend_gini = add_trend_trace(df_gini['Gini_index'], df_gini['Happiness'], 'blue', True)
trend_gdp  = add_trend_trace(df_gdp['GDP_current_usd'], df_gdp['Happiness'], 'blue', False)
trend_gni  = add_trend_trace(df_gni['GNI_current_usd'], df_gni['Happiness'], 'blue', False)

fig = go.Figure(data=[trace_gini, trace_gdp, trace_gni, trend_gini, trend_gdp, trend_gni])

steps = [
    dict(method='update', label='Gini Index',
         args=[{'visible': [True, False, False, True, False, False]},
               {'title': 'Happiness vs Gini Index',
                'xaxis': {'title': 'Gini Index', 'range': x_range_gini, 'fixedrange': True},
                'yaxis': {'title': 'Happiness', 'range': y_range, 'fixedrange': True}}]),
    dict(method='update', label='GDP per Capita',
         args=[{'visible': [False, True, False, False, True, False]},
               {'title': 'Happiness vs GDP per Capita',
                'xaxis': {'title': 'GDP per Capita (USD)', 'range': x_range_gdp, 'fixedrange': True},
                'yaxis': {'title': 'Happiness', 'range': y_range, 'fixedrange': True}}]),
    dict(method='update', label='GNI per Capita',
         args=[{'visible': [False, False, True, False, False, True]},
               {'title': 'Happiness vs GNI per Capita',
                'xaxis': {'title': 'GNI per Capita (USD)', 'range': x_range_gni, 'fixedrange': True},
                'yaxis': {'title': 'Happiness', 'range': y_range, 'fixedrange': True}}])
]

fig.update_layout(
    paper_bgcolor='#EDF1EC',
    plot_bgcolor='#EDF1EC',
    sliders=[dict(
        active=0,
        currentvalue={'prefix': 'Metric: '},
        pad={'t': 50},
        steps=steps,
        len=0.9,
        x=0.1,
        xanchor='left'
    )],
    title="Happiness vs Gini Index",
    title_font_color='black',
    width=800,
    height=600,
    autosize=False,
    margin=dict(t=80, b=80, l=80, r=80),
    xaxis=dict(
        title=dict(text='Gini Index', font=dict(size=14, color='black')),
        range=x_range_gini,
        fixedrange=True,
        tickfont=dict(size=12, color='black'),
        automargin=False
    ),
    yaxis=dict(
        title=dict(text='Happiness', font=dict(size=14, color='black')),
        range=y_range,
        fixedrange=True,
        tickmode='linear',
        dtick=1,
        tickfont=dict(size=12, color='black'),
        automargin=False,
        autorange='reversed'
    ),
    hoverlabel=dict(font=dict(size=12, color='black')),
    showlegend=True,
    legend=dict(
        font=dict(size=12, color='black'),
        x=0.01, y=0.99,
        borderwidth=0,
    )
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('final.csv')
df['Country'] = df['Country'].str.strip()

def prepare_rankings(df, metric_col, ascending_rank):
    df_filtered = df.dropna(subset=[metric_col, 'Happiness']).copy()
    df_filtered = df_filtered.sort_values(by=[metric_col, 'Country'], ascending=[ascending_rank, True])
    df_filtered[f'{metric_col} Rank'] = df_filtered[metric_col].rank(method='first', ascending=ascending_rank).astype(int)
    df_filtered['Happiness Rank'] = df_filtered['Happiness'].rank(method='min', ascending=False).astype(int)
    df_filtered['Mismatch'] = (df_filtered[f'{metric_col} Rank'] - df_filtered['Happiness Rank']).abs()
    return df_filtered

def create_slope_traces(df_metric, metric_col, display_name, mismatch_threshold):
    traces = []
    for _, row in df_metric.iterrows():
        color = '#1E88E5' if row['Mismatch'] <= mismatch_threshold else '#D81B60'
        hovertext = (
            f"{row['Country']}<br>"
            f"{display_name} Rank: {row[f'{metric_col} Rank']}<br>"
            f"Happiness Rank: {row['Happiness Rank']}<br>"
            f"Mismatch: {row['Mismatch']}"
        )
        traces.append(go.Scatter(
            x=[f'{display_name} Rank', 'Happiness Rank'],
            y=[row[f'{metric_col} Rank'], row['Happiness Rank']],
            mode='lines+markers',
            line=dict(color=color, width=2),
            name=row['Country'],
            hoverinfo='text',
            hovertext=hovertext
        ))
    return traces

df_gini = prepare_rankings(df, 'Gini_index', ascending_rank=True)
df_gdp = prepare_rankings(df, 'GDP_current_usd', ascending_rank=False)
df_gni = prepare_rankings(df, 'GNI_current_usd', ascending_rank=False)

red_gini = (df_gini['Mismatch'] > 15).sum()
red_gdp = (df_gdp['Mismatch'] > 30).sum()
red_gni = (df_gni['Mismatch'] > 30).sum()

traces_gini = create_slope_traces(df_gini, 'Gini_index', 'Gini Index', mismatch_threshold=15)
traces_gdp = create_slope_traces(df_gdp, 'GDP_current_usd', 'GDP', mismatch_threshold=30)
traces_gni = create_slope_traces(df_gni, 'GNI_current_usd', 'GNI', mismatch_threshold=30)

fig = go.Figure(data=traces_gini)
for trace in traces_gdp + traces_gni:
    trace.visible = False
    fig.add_trace(trace)

steps = []
total_traces = len(traces_gini) + len(traces_gdp) + len(traces_gni)
start_idx = 0
metrics = [
    ('Gini Index', traces_gini, 20, red_gini),
    ('GDP', traces_gdp, 30, red_gdp),
    ('GNI', traces_gni, 30, red_gni)
]

for metric_name, traces_metric, threshold, red_count in metrics:
    visible = [False] * total_traces
    for i in range(len(traces_metric)):
        visible[start_idx + i] = True
    step = dict(
        method='update',
        label=metric_name,
        args=[
            {'visible': visible},
            {'title': f'{metric_name} vs Happiness Rankings',
             'xaxis': {'title': '', 'tickvals': [f'{metric_name} Rank', 'Happiness Rank']},
             'yaxis': {'title': 'Rank (1 = Best)', 'autorange': 'reversed'},
             'annotations': [dict(
                 x=1.05,
                 y=1.05,
                 xref='paper',
                 yref='paper',
                 showarrow=False,
                 align='left',
                 bgcolor='#D81B60',
                 font=dict(color='white', size=12),
                 text=f'Mismatch > {threshold}:<br><b>{red_count}</b>',
                 bordercolor='black',
                 borderwidth=1
             )]
            }
        ]
    )
    steps.append(step)
    start_idx += len(traces_metric)

fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={"prefix": "Metric: "},
        pad={"t": 50},
        steps=steps,
        len=0.9,
        x=0.1,
        xanchor='left'
    )],
    yaxis=dict(autorange='reversed'),
    showlegend=False,
    height=800,
    width=950,
    margin=dict(t=80, b=80, l=80, r=100),
    title='Gini Index vs Happiness Rankings',
    xaxis={'title': '', 'tickvals': ['Gini Index Rank', 'Happiness Rank']},
    annotations=[dict(
        x=1.05,
        y=1.05,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        bgcolor='#D81B60',
        font=dict(color='white', size=12),
        text=f'Mismatch > 20:<br><b>{red_gini}</b>',
        bordercolor='black',
        borderwidth=1
    )]
)

fig.show()

This figure shows a graph with the ranking of Gini index, GDP and GNI compared to the happiness. This way a mismatch can be seen and the amount of extreme cases is visible. When the mismatch is >30, the line turns red. For the gini index this is slightly difference, the line is red when the mismatch is >20. This is because there is a lot of missing data for gini index, so the sample rate is smaller. Therefore a mismatch of >30 has a way bigger impact than it has for GDP or GNI.

**Money does not buy happiness**

Economic wealth could explain some of the variation in national happiness. However, other societal indicators may offer different insights. To further investigate the what influences the way people perceive their happiness, we examined its relationship with education expenditure, unemployment rate, and life expectancy.

In the plot about Happiness vs Education Expenditure, there is a slight positive trend. Countries that allocate a higher percentage of their GDP to education generally report higher happiness levels according to the dataset. It suggests that investment in quality education contributes to societal well-being and happiness.

The plot about Happiness vs Unemployment Rate has a slight negative trendline: as unemployment increases, average happiness declines according to the dataset. This is a result in line with previous research like Winkelmann (2014), which found that unemployment consistently and significantly reduces life satisfaction and caused a reduction in the happiness score recorded, both due to economic insecurity and the psychological toll of uncertainty.

In the plot about Happiness vs Life Expectancy, a positive correlation can be read from the regression line. Countries where people live longer also tend to report higher happiness scores. This relationship is likely tied to broader indicators of public health and quality of life. As with education, longer life expectancy reflects better healthcare systems, better nutrition, and a better environments, all of this tends to go hand-in-hand with happiness.


In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

df = pd.read_csv('final.csv')

df['Happiness_rank'] = df['Happiness'].rank(method='min', ascending=False).astype(int)
total_countries = df['Country'].nunique()

df_life = df.dropna(subset=['Life_expectancy', 'Happiness', 'Country', 'Happiness_rank'])
df_unemp = df.dropna(subset=['Unemployment_rate', 'Happiness', 'Country', 'Happiness_rank'])
df_edu = df.dropna(subset=['Edu_expenditure_pct', 'Happiness', 'Country', 'Happiness_rank'])

x_range_life = [df_life['Life_expectancy'].min(), df_life['Life_expectancy'].max()]
x_range_unemp = [df_unemp['Unemployment_rate'].min(), df_unemp['Unemployment_rate'].max()]
x_range_edu = [df_edu['Edu_expenditure_pct'].min(), df_edu['Edu_expenditure_pct'].max()]
y_range = [df['Happiness'].min(), df['Happiness'].max()]

def add_trend_trace(x, y, color, visible=True):
    x_vals = x.values
    y_vals = y.values
    slope, intercept = np.polyfit(x_vals, y_vals, 1)
    x_trend = np.linspace(min(x_vals), max(x_vals), 100)
    y_trend = slope * x_trend + intercept
    return go.Scatter(
        x=x_trend,
        y=y_trend,
        mode='lines',
        line=dict(color=color, dash='dash'),
        name='Trend Line',
        visible=visible,
        hoverinfo='skip'
    )

trace_life = go.Scatter(
    x=df_life['Life_expectancy'],
    y=df_life['Happiness'],
    mode='markers',
    name='Life Expectancy',
    visible=True,
    text=df_life['Country'],
    customdata=df_life['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Life Expectancy: %{x:.1f} years<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_unemp = go.Scatter(
    x=df_unemp['Unemployment_rate'],
    y=df_unemp['Happiness'],
    mode='markers',
    name='Unemployment Rate',
    visible=False,
    text=df_unemp['Country'],
    customdata=df_unemp['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Unemployment Rate: %{x:.1f}%<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_edu = go.Scatter(
    x=df_edu['Edu_expenditure_pct'],
    y=df_edu['Happiness'],
    mode='markers',
    name='Education Expenditure',
    visible=False,
    text=df_edu['Country'],
    customdata=df_edu['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Education Expenditure: %{x:.2f}% of GDP<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trend_life = add_trend_trace(df_life['Life_expectancy'], df_life['Happiness'], 'blue', True)
trend_unemp = add_trend_trace(df_unemp['Unemployment_rate'], df_unemp['Happiness'], 'blue', False)
trend_edu = add_trend_trace(df_edu['Edu_expenditure_pct'], df_edu['Happiness'], 'blue', False)

fig = go.Figure(data=[
    trace_life, trace_unemp, trace_edu,
    trend_life, trend_unemp, trend_edu
])

steps = [
    dict(method='update', label='Life Expectancy',
         args=[{'visible': [True, False, False, True, False, False]},
               {'title': 'Happiness vs Life Expectancy',
                'xaxis': {'title': 'Life Expectancy (years)', 'range': x_range_life, 'fixedrange': True},
                'yaxis': {'title': 'Happiness', 'range': y_range, 'fixedrange': True}}]),
    dict(method='update', label='Unemployment Rate',
         args=[{'visible': [False, True, False, False, True, False]},
               {'title': 'Happiness vs Unemployment Rate',
                'xaxis': {'title': 'Unemployment Rate (%)', 'range': x_range_unemp, 'fixedrange': True},
                'yaxis': {'title': 'Happiness', 'range': y_range, 'fixedrange': True}}]),
    dict(method='update', label='Education Expenditure',
         args=[{'visible': [False, False, True, False, False, True]},
               {'title': 'Happiness vs Education Expenditure',
                'xaxis': {'title': 'Education Expenditure (% of GDP)', 'range': x_range_edu, 'fixedrange': True},
                'yaxis': {'title': 'Happiness', 'range': y_range, 'fixedrange': True}}])
]

fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={'prefix': 'Metric: '},
        pad={'t': 50},
        steps=steps,
        len=0.9,
        x=0.1,
        xanchor='left'
    )],
    title="Happiness vs Life Expectancy",
    width=800,
    height=600,
    autosize=False,
    margin=dict(t=80, b=80, l=80, r=80),
    xaxis=dict(
        title=dict(text='Life Expectancy (years)', font=dict(size=14)),
        range=x_range_life,
        fixedrange=True,
        tickfont=dict(size=12)
    ),
    yaxis=dict(
        title=dict(text='Happiness', font=dict(size=14)),
        range=y_range,
        fixedrange=True,
        tickmode='linear',
        dtick=1,
        tickfont=dict(size=12),
    ),
    hoverlabel=dict(font=dict(size=12)),
    showlegend=True,
    legend=dict(
        font=dict(size=12),
        x=0.01, y=0.99,
        borderwidth=0
    )
)

iplot(fig)


In [ ]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('final.csv')
df['Country'] = df['Country'].str.strip()

def prepare_rankings(df, metric_col, ascending_rank):
    df_filtered = df.dropna(subset=[metric_col, 'Happiness']).copy()
    df_filtered = df_filtered.sort_values(by=[metric_col, 'Country'], ascending=[ascending_rank, True])
    df_filtered[f'{metric_col} Rank'] = df_filtered[metric_col].rank(method='first', ascending=ascending_rank).astype(int)
    df_filtered['Happiness Rank'] = df_filtered['Happiness'].rank(method='min', ascending=False).astype(int)
    df_filtered['Mismatch'] = (df_filtered[f'{metric_col} Rank'] - df_filtered['Happiness Rank']).abs()
    return df_filtered

def create_slope_traces(df_metric, metric_col, display_name, mismatch_threshold):
    traces = []
    for _, row in df_metric.iterrows():
        color = '#1E88E5' if row['Mismatch'] <= mismatch_threshold else '#D81B60'
        hovertext = (
            f"{row['Country']}<br>"
            f"{display_name} Rank: {row[f'{metric_col} Rank']}<br>"
            f"Happiness Rank: {row['Happiness Rank']}<br>"
            f"Mismatch: {row['Mismatch']}"
        )
        traces.append(go.Scatter(
            x=[f'{display_name} Rank', 'Happiness Rank'],
            y=[row[f'{metric_col} Rank'], row['Happiness Rank']],
            mode='lines+markers',
            line=dict(color=color, width=2),
            name=row['Country'],
            hoverinfo='text',
            hovertext=hovertext
        ))
    return traces

df_life = prepare_rankings(df, 'Life_expectancy', ascending_rank=False)
df_unemp = prepare_rankings(df, 'Unemployment_rate', ascending_rank=True)
df_edu = prepare_rankings(df, 'Edu_expenditure_pct', ascending_rank=False)

red_life = (df_life['Mismatch'] > 30).sum()
red_unemp = (df_unemp['Mismatch'] > 30).sum()
red_edu = (df_edu['Mismatch'] > 30).sum()

traces_life = create_slope_traces(df_life, 'Life_expectancy', 'Life expectancy', mismatch_threshold=15)
traces_unemp = create_slope_traces(df_unemp, 'Unemployment_rate', 'Unemployment rate', mismatch_threshold=30)
traces_edu = create_slope_traces(df_edu, 'Edu_expenditure_pct', 'Education expenditure', mismatch_threshold=30)

fig = go.Figure(data=traces_life)
for trace in traces_unemp + traces_edu:
    trace.visible = False
    fig.add_trace(trace)

steps = []
total_traces = len(traces_life) + len(traces_unemp) + len(traces_edu)
start_idx = 0
metrics = [
    ('Life expectancy', traces_life, 30, red_life),
    ('Unemployment rate', traces_unemp, 30, red_unemp),
    ('Education expenditure', traces_edu, 30, red_edu)
]

for metric_name, traces_metric, threshold, red_count in metrics:
    visible = [False] * total_traces
    for i in range(len(traces_metric)):
        visible[start_idx + i] = True
    step = dict(
        method='update',
        label=metric_name,
        args=[
            {'visible': visible},
            {'title': f'{metric_name} vs Happiness Rankings',
             'xaxis': {'title': '', 'tickvals': [f'{metric_name} Rank', 'Happiness Rank']},
             'yaxis': {'title': 'Rank (1 = Best)', 'autorange': 'reversed'},
             'annotations': [dict(
                 x=1.05,
                 y=1.05,
                 xref='paper',
                 yref='paper',
                 showarrow=False,
                 align='left',
                 bgcolor='#D81B60',
                 font=dict(color='white', size=12),
                 text=f'Mismatch > {threshold}:<br><b>{red_count}</b>',
                 bordercolor='black',
                 borderwidth=1
             )]
            }
        ]
    )
    steps.append(step)
    start_idx += len(traces_metric)

fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={"prefix": "Metric: "},
        pad={"t": 50},
        steps=steps,
        len=0.9,
        x=0.1,
        xanchor='left'
    )],
    yaxis=dict(autorange='reversed'),
    showlegend=False,
    height=800,
    width=950,
    margin=dict(t=80, b=80, l=80, r=100),
    title='Life expectancy vs Happiness Rankings',
    xaxis={'title': '', 'tickvals': ['Life expectancy Rank', 'Happiness Rank']},
    annotations=[dict(
        x=1.05,
        y=1.05,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        bgcolor='#D81B60',
        font=dict(color='white', size=12),
        text=f'Mismatch > 30:<br><b>{red_life}</b>',
        bordercolor='black',
        borderwidth=1
    )]
)

fig.show()

In [5]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('final.csv')

fig = px.choropleth(
    df,
    locations='Country',
    locationmode='country names',
    color='Happiness',
    hover_name='Country',
    color_continuous_scale='Viridis',
    range_color=(1, 10),
    title='World Happiness Score by Country',
    width=750,
    height=500
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth',
        showocean=True,
        oceancolor='lightblue',
        bgcolor='#EAF4F4'              
    ),
    coloraxis_colorbar=dict(
        tickvals=[1, 4, 7, 10],
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5,
        thickness=15,
        ticklen=0,
        outlinewidth=1,
        outlinecolor='black',
        bgcolor='#EAF4F4'
    ),
    title_x=0.5,
    paper_bgcolor='#EAF4F4',
    plot_bgcolor='#EAF4F4',
    margin={"r":0,"t":60,"l":0,"b":0}
)

fig.show()


In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df = pd.read_csv('final.csv')
others = [
    'GDP_current_usd',
    'GNI_current_usd',
    'Gini_index',
    'Life_expectancy',
    'Unemployment_rate',
    'Edu_expenditure_pct'
]

corr_with_hap = df[others].corrwith(df['Happiness'])
corr_with_hap = corr_with_hap.reindex(
    corr_with_hap.abs().sort_values(ascending=False).index
)

fig = go.Figure(go.Heatmap(
    z=[corr_with_hap.values],
    x=corr_with_hap.index,
    y=['Happiness'],
    zmin=-1, zmid=0, zmax=1,
    colorscale='RdBu',
    colorbar=dict(title='Pearson r'),
    text=[[f"{v:.2f}" for v in corr_with_hap.values]],
    texttemplate='%{text}',
    textfont=dict(size=14),
    hoverongaps=False
))

fig.update_layout(
    title="Correlations: Happiness vs Other Indicators",
    xaxis_tickangle=-45,
    yaxis=dict(tickfont=dict(size=14)),
    height=500,
    margin=dict(l=80, r=80, t=60, b=100),
    paper_bgcolor='#EDF1EC',
    plot_bgcolor='#EDF1EC'
)

fig.show()


In [7]:
import plotly.graph_objects as go

factors = [
    'Life expectancy',
    'Education expenditure',
    'Unemployment rate',
    'Gini index',
    'GDP in current USD',
    'GNI in current USD'
]
correlations = [0.77, 0.32, -0.25, -0.22, 0.18, 0.17]

abs_vals = [abs(r) for r in correlations]
total = sum(abs_vals)
percentages = [v / total * 100 for v in abs_vals]

bases = [0]
for pct in percentages[:-1]:
    bases.append(bases[-1] + pct)

colors = [
    '#256caf',
    '#abd2e5',
    '#fbceb6',
    '#fcd5bf',
    '#d6e7f1',
    '#d6e7f1'
]

fig = go.Figure()

for factor, pct, base, color in zip(factors, percentages, bases, colors):
    fig.add_trace(go.Bar(
        x=[pct],
        y=['Effect on Happiness (%)'],
        name=factor,
        orientation='h',
        marker_color=color,
        base=[base],
        hovertemplate=f'{factor}: {pct:.1f}%<extra></extra>'
    ))

socio_end = bases[3]
econ_start = socio_end
econ_end = bases[-1] + percentages[-1]

fig.add_shape(type="rect",
    x0=0, x1=socio_end,
    y0=-0.5, y1=0.5,
    xref='x', yref='y',
    line=dict(color='black', width=2),
    fillcolor='rgba(0,0,0,0)'
)
fig.add_shape(type="rect",
    x0=econ_start, x1=econ_end,
    y0=-0.5, y1=0.5,
    xref='x', yref='y',
    line=dict(color='black', width=2),
    fillcolor='rgba(0,0,0,0)'
)

fig.add_annotation(
    x=socio_end / 2,
    y=0.6,
    text="Socioeconomic 70.2%",
    showarrow=False,
    font=dict(size=12)
)
fig.add_annotation(
    x=(econ_start + econ_end) / 2,
    y=0.6,
    text="Economic 29.8%",
    showarrow=False,
    font=dict(size=12)
)

fig.update_layout(
    barmode='stack',
    title="Tug-of-War: Relative Effect % on Happiness",
    xaxis_title="Percent of Total Effect",
    template="simple_white",
    showlegend=True,
    legend_title="Factors",
    height=350,
    margin=dict(l=20, r=20, t=60, b=20),
    paper_bgcolor='#EDF1EC',
    plot_bgcolor='#EDF1EC'
)

fig.show()
